# Table of Contents

[01. Import Library](#01)<br>

[02. Load Data](#02)<br>

[03. Exploratory Data Analysis (EDA)](#03)<br>

&nbsp;&nbsp;&nbsp;[3.1. Dependent Variable](#3.1)<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[3.1.1. Use matplotlib](#3.1.1)<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[3.1.2. Use plotly_express](#3.1.2)<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[3.1.3. Use seaborn](#3.1.3)<br>

&nbsp;&nbsp;&nbsp;[3.2. Independent Variables](#3.2)<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[3.2.1. Outliers](#3.2.1)<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[3.2.2. Quantitative - Qualitative Features](#3.2.2)<br>

[04. Prediction](#04)<br>

&nbsp;&nbsp;&nbsp;[4.1. Baseline Model](#4.1)<br>
&nbsp;&nbsp;&nbsp;[4.2. Feature Engineering](#4.2)<br>
&nbsp;&nbsp;&nbsp;[4.3. Feature Scaling](#4.3)<br>


[05. Results](#05)<br>



In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
for dirname, _, filenames in os.walk('/kaggle'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


# 01. Import Library<a id='01'></a>

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()
sns.set_style('darkgrid')

import plotly_express as px

from scipy import stats
from scipy.stats import norm, skew 

import pandas_profiling

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor


from lightgbm import LGBMRegressor

# 02. Load Data <a id='02'></a>

In [ ]:
print("List of files:", os.listdir('/kaggle/input/house-prices-advanced-regression-techniques'))

# Train data
df_train = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
print("\nTrain data length:",df_train.shape)
print("\nTrain data columns:",df_train.columns)
print("\nTrain data columns:",df_train.info())
print("\nTrain data:\n\n",df_train.head())

In [ ]:
# Test data
df_test = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')
print("\nTest data length:",df_test.shape)

# 03. Exploratory Data Analysis (EDA)<a id='03'></a>

In [ ]:
# Correlation
df_train_corr = df_train.corr()
df_train_corr

In [ ]:
df_train_corr.style.background_gradient(cmap='coolwarm', axis=None)

In [ ]:
# SalePrice has highest corr with OverallQual
df_train_corr[['SalePrice','OverallQual']].style.background_gradient(cmap='coolwarm', axis=None)

In [ ]:
# Use panda profile report
# df_train.profile_report()

## 3.1. Dependent Variable<a id='3.1'></a>

In [ ]:
df_train['SalePrice'].describe()

In [ ]:
ax=df_train['SalePrice'].plot.hist(bins=100, alpha=0.6)

### 3.1.1. Use matplotlib<a id='3.1.1'></a>

In [ ]:
# Use matplotlib

# plt.style.use('ggplot')
plt.hist(df_train['SalePrice'], bins = 100)

# Add title and axis names
plt.title('Sales Price')
plt.xlabel('Price')
plt.ylabel('Frequency') 

plt.show()

In [ ]:
# Scatter Plot
fig, ax = plt.subplots()
ax.scatter(df_train['GrLivArea'], df_train['SalePrice'])
plt.xlabel('GrLivArea', fontsize=12)
plt.ylabel('SalePrice', fontsize=12)
plt.title('Sale Price', fontsize=16)
plt.show()

In [ ]:
# QQ-plot
fig = plt.figure()
ax = fig.add_subplot()
res = stats.probplot(df_train['SalePrice'], plot=plt)
plt.show()

### 3.1.2. Use plotly_express<a id='3.1.2'></a>

In [ ]:
# Scatter Plot with color from 2nd variable
px.scatter(df_train, x='GrLivArea', y='SalePrice', color='OverallQual')

In [ ]:
# Scatter Plot with color from 2nd variable
px.scatter(df_train, x='TotalBsmtSF', y='SalePrice', color='OverallQual')

In [ ]:
# Box Plot
px.box(df_train[['OverallQual', 'SalePrice']].sort_values(by='OverallQual')
       , x='OverallQual'
       , y='SalePrice'
       , color='OverallQual')

In [ ]:
# Box Plot
px.box(df_train[['SaleCondition', 'SalePrice']].sort_values(by='SaleCondition')
       , x='SaleCondition'
       , y='SalePrice'
       , color='SaleCondition')

In [ ]:
# Box Plot
px.box(df_train[['ExterQual', 'SalePrice']].sort_values(by='ExterQual')
       , x='ExterQual'
       , y='SalePrice'
       , color='ExterQual')

### 3.1.3. Use seaborn<a id='3.1.3'></a>

In [ ]:
sns.distplot(df_train['SalePrice'] , fit=norm);

# Get the fitted parameters used by the function
(mu, sigma) = norm.fit(df_train['SalePrice'])
print( '\n mu = {:.2f} and sigma = {:.2f}\n'.format(mu, sigma))

# Plot the distribution
plt.legend(['Normal dist. ($\mu=$ {:.2f} and $\sigma=$ {:.2f} )'.format(mu, sigma)],
            loc='upper right')

ax = plt.axes()
plt.ylabel('Frequency')
plt.title('SalePrice distribution')


In [ ]:
sns.heatmap(df_train_corr, 
            xticklabels=df_train_corr.columns.values,
            yticklabels=df_train_corr.columns.values)

## 3.2. Independent Variables<a id='3.2'></a>

In [ ]:
df_train.drop(['SalePrice'], axis = 1).describe().T

### 3.2.1. Outliers<a id='3.2.1'></a>

In [ ]:
# Clean outliers
print("Length of data before dropping outliers:", len(df_train))
df_train = df_train.drop(df_train[(df_train['GrLivArea']>4000) 
                                & (df_train['SalePrice']<300000)].index)
print("Length of data after dropping outliers:", len(df_train))
df_train = df_train.drop(df_train[(df_train['GrLivArea']>5000) 
                                | (df_train['SalePrice']>500000)].index)
print("Length of data after dropping outliers:", len(df_train))

### 3.2.2. Quantitative - Qualitative Features<a id='3.2.2'></a>

In [ ]:
# Quantitative Variables
quan_var = [q for q in df_train.columns if df_train.dtypes[q] != 'object']
quan_var.remove('SalePrice') 
quan_var.remove('Id')
print("Quantitative Variables:\n", quan_var)

# Qualitative Variables
qual_var = [q for q in df_train.columns if df_train.dtypes[q] == 'object']
print("\nQualitative Variables:\n", qual_var)

In [ ]:
# Combine all data
ntrain = df_train.shape[0]
ntest = df_test.shape[0]
y_train = df_train.SalePrice.values
df_all_data = pd.concat((df_train, df_test)).reset_index(drop=True)
df_all_data.drop(['Id','SalePrice'], axis=1, inplace=True)
print("all_data size is : {}".format(df_all_data.shape))

# Calculate missing data ratio
df_all_data_na = (df_all_data.isnull().sum() / len(df_all_data)) * 100
df_all_data_na = df_all_data_na.drop(df_all_data_na[df_all_data_na == 0].index).sort_values(ascending=False)[:50]
missing_data = pd.DataFrame({'Missing Ratio' :df_all_data_na})
print('Missing data percentage:\n',missing_data.head(50))

# Plot
f, ax = plt.subplots(figsize=(15, 12))
plt.xticks(rotation='90')
ax.set_facecolor("white")
sns.barplot(x=df_all_data_na.index, y=df_all_data_na)
sns.color_palette('pastel')
plt.xlabel('Features', fontsize=12)
plt.ylabel('Percent of missing values', fontsize=12)
plt.title('Percent missing data by feature', fontsize=15)

In [ ]:
df_all_data

# 04. Prediction <a id='04'></a>

In [ ]:
df_result = pd.DataFrame(columns=['Model','RMSE','MSE','Summary'])
print(df_result)

## 4.1. Baseline Model <a id='4.1'></a>

### Linear Regression

In [ ]:
# Run Linear Regression on a single variable that has the highest corr with dependent variable
X = df_train[['OverallQual']]
y = df_train['SalePrice']

# Train Test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Linear Regression Model
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)

# RMSE
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("Root Mean Squared Error: {:.2f}".format(rmse))
df_result = df_result.append(pd.DataFrame([['Linear Regression'
                                            , rmse
                                            , mse
                                            ,'Baseline model'                               
                                           ]], columns=df_result.columns))
print(df_result)

### Random Forest

In [ ]:
# RandomForestRegressor
rf = RandomForestRegressor(random_state=10)
rf.fit(X_train,y_train)
y_pred_rf = rf.predict(X_test)
mse = mean_squared_error(y_test, y_pred_rf)
rmse = np.sqrt(mean_squared_error(y_test, y_pred_rf))
print("Root Mean Squared Error: {:.2f}".format(rmse))
df_result = df_result.append(pd.DataFrame([['RandomForestRegressor'
                                            , rmse
                                            , mse
                                            ,'Baseline model'                               
                                           ]], columns=df_result.columns))
print(df_result)

### Features - Missing Ratio

In [ ]:
# Get the list of variable based on missing data ratio
features_for_reg = missing_data[missing_data['Missing Ratio']<70].index.values.tolist()


# Get Dummies
X_all = pd.get_dummies(df_all_data[features_for_reg])
X_all.fillna(0, inplace=True)

X = X_all[0:len(df_train)]
y = df_train['SalePrice']

# Initiate train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

rf = RandomForestRegressor(random_state=3)
rf.fit(X_train,y_train)
y_pred_rf = rf.predict(X_test)
mse = mean_squared_error(y_test, y_pred_rf)
rmse = np.sqrt(mean_squared_error(y_test, y_pred_rf))
print("Root Mean Squared Error: {:.2f}".format(rmse))
df_result = df_result.append(pd.DataFrame([['RandomForestRegressor'
                                            , rmse
                                            , mse
                                            ,'Features with less than 70% missing data'                               
                                           ]], columns=df_result.columns))



# Calculate feature importances
importances = rf.feature_importances_
# Sort feature importances in descending order
indices = np.argsort(importances)[::-1]
# Rearrange feature names so they match the sorted feature importances
names = [X_train.columns[i] for i in indices]

print("Most important:\n", names[:10])
print("Least important:\n", names[(-10):])


### Features - Importance

In [ ]:
# Get the list of variable based on rf feature importance
n_features = 45
features_for_reg = names[:n_features]


# Run Linear Regression
X_all = X_all[features_for_reg]
X_all.fillna(0, inplace=True)

X = X_all[0:len(df_train)]
y = df_train['SalePrice']

# Initiate train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

rf = RandomForestRegressor(random_state=3)
rf.fit(X_train,y_train)
y_pred_rf = rf.predict(X_test)
mse = mean_squared_error(y_test, y_pred_rf)
rmse = np.sqrt(mean_squared_error(y_test, y_pred_rf))
print("Root Mean Squared Error: {:.2f}".format(rmse))
df_result = df_result.append(pd.DataFrame([['RandomForestRegressor'
                                            , rmse
                                            , mse
                                            ,'Important features based on RF'                               
                                           ]], columns=df_result.columns))



# Calculate feature importances
importances = rf.feature_importances_
# Sort feature importances in descending order
indices = np.argsort(importances)[::-1]
# Rearrange feature names so they match the sorted feature importances
names = [X_train.columns[i] for i in indices]

print("Most important:\n", names[:10])
print("Least important:\n", names[(-10):])

## 4.2. Feature Engineering <a id='4.2'></a>

In [ ]:
# New feature
df_all_data["OverallQual_Garage_GrLivArea"] = df_all_data["OverallQual"] * df_all_data["GarageArea"] * df_all_data["GrLivArea"]

# Get Dummies
X_all = pd.get_dummies(df_all_data)
X_all.fillna(0, inplace=True)

X = X_all[0:len(df_train)]
y = df_train['SalePrice']

# Initiate train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

rf = RandomForestRegressor(random_state=3)
rf.fit(X_train,y_train)
y_pred_rf = rf.predict(X_test)
mse = mean_squared_error(y_test, y_pred_rf)
rmse = np.sqrt(mean_squared_error(y_test, y_pred_rf))
print("Root Mean Squared Error: {:.2f}".format(rmse))
df_result = df_result.append(pd.DataFrame([['RandomForestRegressor'
                                            , rmse
                                            , mse
                                            ,'Features engineering'                               
                                           ]], columns=df_result.columns))



# Calculate feature importances
importances = rf.feature_importances_
# Sort feature importances in descending order
indices = np.argsort(importances)[::-1]
# Rearrange feature names so they match the sorted feature importances
names = [X_train.columns[i] for i in indices]

print("Most important:\n", names[:10])
print("Least important:\n", names[(-10):])


### Feature Engineering and Missing Value

In [ ]:
# Get the list of variable based on missing data ratio
features_to_drop = missing_data[missing_data['Missing Ratio']>=70].index.values.tolist()


# Get Dummies
X_all = pd.get_dummies(df_all_data[df_all_data.columns.difference(features_to_drop)])
X_all.fillna(0, inplace=True)

X = X_all[0:len(df_train)]
y = df_train['SalePrice']

# Initiate train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

rf = RandomForestRegressor(random_state=3)
rf.fit(X_train,y_train)
y_pred_rf = rf.predict(X_test)
mse = mean_squared_error(y_test, y_pred_rf)
rmse = np.sqrt(mean_squared_error(y_test, y_pred_rf))
print("Root Mean Squared Error: {:.2f}".format(rmse))
df_result = df_result.append(pd.DataFrame([['RandomForestRegressor'
                                            , rmse
                                            , mse
                                            ,'Features Engineering & < 70% missing data'                               
                                           ]], columns=df_result.columns))



# Calculate feature importances
importances = rf.feature_importances_
# Sort feature importances in descending order
indices = np.argsort(importances)[::-1]
# Rearrange feature names so they match the sorted feature importances
names = [X_train.columns[i] for i in indices]

print("Most important:\n", names[:10])
print("Least important:\n", names[(-10):])



### Light GBM

In [ ]:
lgb_model = LGBMRegressor().fit(X_train, y_train)
y_pred_lgb = lgb_model.predict(X_test)

mse = mean_squared_error(y_test, y_pred_lgb)
rmse = np.sqrt(mean_squared_error(y_test, y_pred_lgb))
print("Root Mean Squared Error: {:.2f}".format(rmse))
df_result = df_result.append(pd.DataFrame([['LGBMRegressor'
                                            , rmse
                                            , mse
                                            ,'Features Engineering & < 70% missing data'                               
                                           ]], columns=df_result.columns))

### Light GBM best params

In [ ]:
# Grid search (narrow down to this grid after several iterations)
lgb_params = {"learning_rate": [0.005, 0.01],
               "n_estimators": [5000],
               "max_depth": [4, 5],
               "feature_fraction": [0.1, 0.2, 0.3],
               "colsample_bytree": [0.8],
               'num_leaves': [4, 5]}
                              
lgb_cv_model = GridSearchCV(lgb_model,
                             lgb_params,
                             cv=10,
                             n_jobs=-1,
                             verbose=2).fit(X_train, y_train)

print(lgb_cv_model.best_params_)

# use best params
lgb_tuned = LGBMRegressor(**lgb_cv_model.best_params_).fit(X_train, y_train)
y_pred_lgb = lgb_tuned.predict(X_test)

mse = mean_squared_error(y_test, y_pred_lgb)
rmse = np.sqrt(mean_squared_error(y_test, y_pred_lgb))
print("Root Mean Squared Error: {:.2f}".format(rmse))

df_result = df_result.append(pd.DataFrame([['LGBMRegressor'
                                            , rmse
                                            , mse
                                            ,'Tuned model with Features Engineering & < 70% missing data'                               
                                           ]], columns=df_result.columns))

### Gradient Boosting Regressor

In [ ]:
gb_model = GradientBoostingRegressor()
gb_model.fit(X_train, y_train)
y_pred_gb = gb_model.predict(X_test)

mse = mean_squared_error(y_test, y_pred_gb)
rmse = np.sqrt(mean_squared_error(y_test, y_pred_gb))
print("Root Mean Squared Error: {:.2f}".format(rmse))

df_result = df_result.append(pd.DataFrame([['GradientBoostingRegressor'
                                            , rmse
                                            , mse
                                            ,'Features Engineering & < 70% missing data'                               
                                           ]], columns=df_result.columns))

### GB with Best Params

In [ ]:
# Grid search (narrow down to this grid after several iterations)
gb_params = {'n_estimators': [1000,5000],
             'max_depth': [4,5],
             'min_samples_split': [3,5],
             'learning_rate': [0.005, 0.01],
             'loss': ['ls']}
                              
gb_cv_model = GridSearchCV(gb_model,
                           gb_params,
                           cv=10,
                           n_jobs=-1,
                           verbose=2).fit(X_train, y_train)

print(gb_cv_model.best_params_)

# use best params
gb_tuned = GradientBoostingRegressor(**gb_cv_model.best_params_).fit(X_train, y_train)
y_pred_gb = gb_tuned.predict(X_test)

mse = mean_squared_error(y_test, y_pred_gb)
rmse = np.sqrt(mean_squared_error(y_test, y_pred_gb))
print("Root Mean Squared Error: {:.2f}".format(rmse))

df_result = df_result.append(pd.DataFrame([['GradientBoostingRegressor'
                                            , rmse
                                            , mse
                                            ,'Tuned & Features Engineering & < 70% missing data'                               
                                           ]], columns=df_result.columns))

## 4.3. Feature Scaling <a id='4.3'></a>

In [ ]:
df_all_data.describe().T

In [ ]:
df_all_data_scaled =  pd.DataFrame()

scaler = StandardScaler()

for col in quan_var:    
    scaler.fit((np.array(df_all_data[col])).reshape(-1, 1))
    scaled_list = scaler.transform((np.array(df_all_data[col])).reshape(-1, 1))
    
    # Convert list of list to flat list before putting back to the df
    df_all_data_scaled[col] = [item for elem in scaled_list for item in elem]
    
df_all_data_scaled.describe().T


In [ ]:
# New feature
df_all_data_scaled["OverallQual_Garage_GrLivArea"] = df_all_data_scaled["OverallQual"] * \
                                                     df_all_data_scaled["GarageArea"] * \
                                                     df_all_data_scaled["GrLivArea"]

# Get Dummies
X_all = pd.get_dummies(df_all_data_scaled)
X_all.fillna(0, inplace=True)

X = X_all[0:len(df_train)]
y = df_train['SalePrice']

# Initiate train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

rf_scaled = RandomForestRegressor(random_state=3)
rf_scaled.fit(X_train,y_train)
y_pred_rf = rf_scaled.predict(X_test)
mse = mean_squared_error(y_test, y_pred_rf)
rmse = np.sqrt(mean_squared_error(y_test, y_pred_rf))
print("Root Mean Squared Error: {:.2f}".format(rmse))
df_result = df_result.append(pd.DataFrame([['RandomForestRegressor'
                                            , rmse
                                            , mse
                                            ,'Features engineering & Scaled'                               
                                           ]], columns=df_result.columns))



# Calculate feature importances
importances = rf_scaled.feature_importances_
# Sort feature importances in descending order
indices = np.argsort(importances)[::-1]
# Rearrange feature names so they match the sorted feature importances
names = [X_train.columns[i] for i in indices]

print("Most important:\n", names[:10])
print("Least important:\n", names[(-10):])


In [ ]:
# Get the list of variable based on missing data ratio
features_to_drop = missing_data[missing_data['Missing Ratio']>=70].index.values.tolist()


# Get Dummies
X_all = pd.get_dummies(df_all_data_scaled[df_all_data_scaled.columns.difference(features_to_drop)])
X_all.fillna(0, inplace=True)

lgb_scaled = LGBMRegressor().fit(X_train, y_train)
y_pred_lgb = lgb_scaled.predict(X_test)

mse = mean_squared_error(y_test, y_pred_lgb)
rmse = np.sqrt(mean_squared_error(y_test, y_pred_lgb))
print("Root Mean Squared Error: {:.2f}".format(rmse))
df_result = df_result.append(pd.DataFrame([['LGBMRegressor'
                                            , rmse
                                            , mse
                                            ,'Features Engineering & Scaled & < 70% missing data'                               
                                           ]], columns=df_result.columns))

In [ ]:
# Grid search (narrow down to this grid after several iterations)
lgb_params = {"learning_rate": [0.005, 0.01],
               "n_estimators": [5000],
               "max_depth": [4, 5],
               "feature_fraction": [0.1, 0.2, 0.3],
               "colsample_bytree": [0.8],
               'num_leaves': [4, 5]}
                              
lgb_cv_model = GridSearchCV(lgb_scaled,
                             lgb_params,
                             cv=10,
                             n_jobs=-1,
                             verbose=2).fit(X_train, y_train)

print(lgb_cv_model.best_params_)

# use best params
lgb_scaled_tuned = LGBMRegressor(**lgb_cv_model.best_params_).fit(X_train, y_train)
y_pred_lgb_scaled_tuned = lgb_scaled_tuned.predict(X_test)

mse = mean_squared_error(y_test, y_pred_lgb_scaled_tuned)
rmse = np.sqrt(mean_squared_error(y_test, y_pred_lgb_scaled_tuned))
print("Root Mean Squared Error: {:.2f}".format(rmse))

df_result = df_result.append(pd.DataFrame([['LGBMRegressor'
                                            , rmse
                                            , mse
                                            ,'Tuned model with Features Engineering & Scaled & < 70% missing data'                               
                                           ]], columns=df_result.columns))

# 05. Results<a id='05'></a>

In [ ]:
df_result['RMSE'] = df_result['RMSE'].astype(int)
df_result['MSE'] = df_result['MSE'].astype(int)
df_result

### Prediction Submission

In [ ]:
# # Predict using rf
# X_test = X_all.iloc[len(df_train):len(X_all)]
# y_pred_rf = rf.predict(X_test)

# # Predict using lgb
# X_test = X_all.iloc[len(df_train):len(X_all)]
# y_pred_lgb = lgb_model.predict(X_test)

# # Predict using lgb_tuned
# X_test = X_all.iloc[len(df_train):len(X_all)]
# y_pred_lgb_tuned = lgb_tuned.predict(X_test)

# Predict using rf_scaled
X_test = X_all.iloc[len(df_train):len(X_all)]
y_pred_rf_scaled = rf_scaled.predict(X_test)

# Predict using lgb_scaled
X_test = X_all.iloc[len(df_train):len(X_all)]
y_pred_lgb_tuned = lgb_scaled_tuned.predict(X_test)

In [ ]:
# Submission
sub = pd.DataFrame()
sub['Id'] = df_test['Id']
sub['SalePrice'] = y_pred_lgb_tuned
sub.to_csv('submission.csv',index=False)